In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [7]:
dataset = pd.read_csv("heart_failure_clinical_records_dataset.csv")

In [33]:
dataset.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [25]:
# Time column is number of days until death (1) or censoring (2)
# For now, we will omit the time column from the rough-draft analysis
X = dataset.iloc[:, 0:11]
y = dataset["DEATH_EVENT"]


C:\Users\Seth\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning:

Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.



In [34]:
# Rescale columns based on min and max. Only need to rescale columns 0, 2, 4, 6, 8
mm_scaler = preprocessing.MinMaxScaler()
X[["age","creatinine_phosphokinase","ejection_fraction","platelets","serum_sodium"]] = mm_scaler.fit_transform(X[["age","creatinine_phosphokinase","ejection_fraction","platelets","serum_sodium"]])

X_train, X_test, y_train, y_test = train_test_split(X_minmax, y, test_size=0.3)

C:\Users\Seth\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning:

Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.



In [36]:
# Apply Logistic regression on training dataset
clf = LogisticRegression()
clf.fit(X_train, y_train)

C:\Users\Seth\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [39]:
# Use fitted model to predict classification based on test set
y_predicted = clf.predict(X_test)

In [44]:
# Get the confusion matrix
cf_results = metrics.confusion_matrix(y_test, y_predicted)
print(cf_results)

[[61  2]
 [23  4]]


In [45]:
# Check accuracy of predictions
accuracy = metrics.accuracy_score(y_test, y_predicted)
print(accuracy)

0.7222222222222222
